In [ ]:
import sys
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install ta
!{sys.executable} -m pip install finta
!{sys.executable} -m pip install mplfinance
!{sys.executable} -m pip install yfinance
!{sys.executable} -m pip install ta-lib
!{sys.executable} -m pip install numba
!{sys.executable} -m pip install bokeh

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from numba import njit, jit 
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, CrosshairTool
from bokeh.io import output_notebook # enables plot interface in J notebook

## Getting Data

### Importing from Yahoo Finance

In [2]:
tsla_data = yf.download('TSLA')
tsla_data.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-05-11,795.000000,809.770020,727.200012,734.000000,734.000000,32408200
2022-05-12,701.000000,759.659973,680.000000,728.000000,728.000000,46771000
2022-05-13,773.479980,787.349976,751.570007,769.590027,769.590027,30651800
2022-05-16,767.159973,769.760010,719.090027,724.369995,724.369995,28699500
2022-05-17,747.359985,764.479980,728.849976,761.609985,761.609985,26243585


### Converting prices from dataframe to a numpy array

In [3]:
Open = tsla_data['Open'].values.astype(np.float32)
High = tsla_data['High'].values.astype(np.float32)
Low = tsla_data['Low'].values.astype(np.float32)
Close = tsla_data['Close'].values.astype(np.float32)
Volume = tsla_data['Volume'].values.astype(np.float32)

## Indicators

### Lag

In [4]:
@njit
def lag(arr, lag):
    """
        Documentation:
         Compute a lagged version of a time series, shifting the time base back by a given number of observations.
        :param arr: An array time series
        :param lag: The number of lags (in units of observations).
        :return: lag array  of the time series.
        """
    lagged = np.roll(arr, lag)
    for i in range(lag):
        lagged[i] = np.NaN
    return lagged

### Bollinger Band

In [5]:
@njit
def bband(arr, window, std_param):
    """
    Bollinger Bands are volatility bands placed above and below a moving average. Volatility is based on
    the standard deviation, which changes as volatility increases and decreases.
    The bands automatically widen when volatility increases and contract when volatility decreases.
    The bands has three components:
    * Middle Band = 20-day simple moving average (SMA)
    * Upper Band = 20-day SMA + (20-day standard deviation of price x 2)
    * Lower Band = 20-day SMA - (20-day standard deviation of price x 2)
    :param arr: the array input, usually the close price
    :param window: the rolling period for the calculations
    :param std_param: the standard deviation multiplier
    :return: np.array vector with the middle, upper and lower bands calculation
    """
    # MA
    ma_values_close = np.array([arr[i:i + window].mean() for i in range(len(arr) - window + 1)])
    ma_close = np.append(np.zeros(window - 1) + np.nan, ma_values_close).astype(np.float32)

    # STD
    np_std_values_close = np.array([arr[i:i + window].std() for i in range(len(arr) - window + 1)])
    std_close = np.append(np.zeros(window - 1) + np.nan, np_std_values_close).astype(np.float32)

    upper_band = ma_close + std_param * std_close
    lower_band = ma_close - std_param * std_close
    middle_band = (upper_band + lower_band) / 2

    return upper_band.astype(np.float32), lower_band.astype(np.float32), middle_band.astype(np.float32)


### True Range

In [6]:
@njit
def tr(high: np.array, low: np.array, close: np.array):
    """
    The average true range (ATR) is a technical analysis indicator that measures market volatility by decomposing
    the entire range of an asset price for that period.
    tr has 3 components, we always save the bigger one as the tr value for that candle, the 3 components are:
    1. current high - current low.
    2. absolute value of the current high - previous close.
    3. absolute value of the current low - previous close.
    :param high:  np.array with the column high. inside values may be np.float32.
    :param low:   np.array with the column low. inside values may be np.float32.
    :param close: np.array with the column close. inside values may be np.float32.
    :return: np.array vector with the values for the true_range.
    """
    true_range = np.zeros(len(high))
    true_range[0] = np.nan

    for i in range(1, len(true_range)):
        high_minus_low = high[i] - low[i]
        abs_high_minus_prev_close = np.fabs(high[i] - close[i - 1])
        abs_low_minus_prev_close = np.fabs(low[i] - close[i - 1])
        true_range[i] = np.amax(np.array([high_minus_low, abs_high_minus_prev_close, abs_low_minus_prev_close]))

    return true_range.astype(np.float32)

### ATR

In [7]:
@njit
def atr(high, low, close, window):
    """
    The Average True Range (ATR) is an indicator that measures volatility,  A volatility formula based only on the
    high-low range would fail to capture volatility from gap or limit moves. Wilder created Average True Range to
    capture this “missing” volatility. It is important to remember that ATR does not provide an indication of price
    direction, just volatility.
    We calculate the ATR by first getting the True range:
    Method 1: Current High less the current Low
    Method 2: Current High less the previous Close (absolute value)
    Method 3: Current Low less the previous Close (absolute value)
    After we got the true range for the series we apply a simple moving average for the true range and by that way we
    get the Average True range of the series
    :param high: The high price in a candle
    :param low: The low price of the candle
    :param close: The closing price for the period
    :param window: the rolling window for the simple moving average
    :return: np.array with the ATR calculations
    """
    true_range = tr(high.astype(np.float32), low.astype(np.float32), close.astype(np.float32))
    np_atr_values = np.array([true_range[i:i + window].mean() for i in range(len(true_range) - window + 1)])
    average_true_range = np.append(np.zeros(window - 1) + np.nan, np_atr_values)

    return average_true_range.astype(np.float32)

### EMA

In [8]:
@njit
def ema(arr, period):
    """
        Documentation:
        An exponential moving average (EMA) is a type of moving average (MA) that places a greater weight and significance
        on the most recent data points. The exponential moving average is also referred to as the exponentially weighted
        moving average. An exponentially weighted moving average reacts more significantly to recent price changes than
        a simple moving average (SMA), which applies an equal weight to all observations in the period.
        EMA(t) = Value t * (Smoothing/(1+days)) + EMA t-1 (1-(Smoothing/(1+days)))
        While there are many possible choices for the smoothing factor, the most common choice is:
        Smoothing = 2
        That gives the most recent observation more weight. If the smoothing factor is increased,
        more recent observations have more influence on the EMA.
        :param arr: numpy array, dtype=np.float32, this is the close time series you want to use
        :param period: integer value, the bigger the value the slower the calculation will but also less impacted by
        outlayers, picking a good value according to the temporality is a most.
        :return: numpy array, dtype=np.float32, this is the value for every moment of the calculation.
        """

    ema_calc = np.zeros(len(arr))
    ema_calc[period - 1] = np.mean(arr[0:period])
    for i in range(period, len(ema_calc)):
        ema_calc[i] = (arr[i] - ema_calc[i - 1]) * (2 / (period + 1)) + ema_calc[i - 1]
    ema_calc[:period - 1] = np.nan

    ema_calc = ema_calc.astype(np.float32)
    return ema_calc


### SMA

In [9]:
@njit
def ma(arr: np.array, window: int):
    """
    ma stands for the moving average of a price series.
    :param arr: np.array with dimension 1, should be np.float32 numbers.
    :param window: int value, its the length of the moving window, minimum value for the window may be 2.
    :return: np.array vector where the first #window-elements are np.nan, then you have the moving average of the arr.
    """
    ma_values_close = np.array([arr[i:i + window].mean() for i in range(len(arr) - window + 1)])
    moving_average = np.append(np.zeros(window - 1) + np.nan, ma_values_close)

    return moving_average.astype(np.float32)


### Keltner Channel

In [10]:
@njit
def keltner_channel(high, low, close, ema_period, multiplier, atr_period):
    """
    Documentation:
    Keltner Channels are volatility-based envelopes set above and below an exponential moving average. Instead of using
    the standard deviation, Keltner Channels use the Average True Range (ATR) to set channel distance. The channels are
    typically set two Average True Range values above and below the 20-day EMA. The exponential moving average dictates
    direction and the Average True Range sets channel width. Keltner Channels are a trend following indicator used to
    identify reversals with channel breakouts and channel direction. Channels can also be used to identify overbought
    and oversold levels when the trend is flat.
    Calculation: There are three steps to calculating Keltner Channels. First, select the length for the exponential
    moving average. Second, choose the time periods for the Average True Range (ATR). Third, choose the multiplier for
    the Average True Range.
    Middle Line: 20-day exponential moving average
    Upper Channel Line: 20-day EMA + (2 x ATR(10))
    Lower Channel Line: 20-day EMA - (2 x ATR(10))
    Interpretation:
    Indicators based on channels, bands and envelopes are designed to encompass most price action. Therefore, moves
    above or below the channel lines warrant attention because they are relatively rare. Trends often start with strong
    moves in one direction or another. A surge above the upper channel line shows extraordinary strength, while a plunge
    below the lower channel line shows extraordinary weakness. Such strong moves can signal the end of one trend and the
    beginning of another.
    :param high: np.array, the high prices of a series
    :param low: np.array, the low prices of a series
    :param close: np.array, the close prices of a series
    :param ema_period: integer, the window period for the moving average
    :param multiplier: integer, the window period for the moving average
    :param atr_period: integer, the window period for the ATR
    :return: 3 numpy array, dtype=np.float32, this is the calculation value.
    """
    # EMA
    middle_line = ema(close.astype(np.float32), np.int32(ema_period))
    upper_channel = middle_line + (multiplier * atr(high, low, close, atr_period))
    lower_channel = middle_line - (multiplier * atr(high, low, close, atr_period))

    return middle_line.astype(np.float32), upper_channel.astype(np.float32), lower_channel.astype(np.float32)

## Lineal Regression

In [11]:
@njit
def lin_reg(y_array, x_array, window):
    # this mask ensures we only calculate using common values (non-nan values)
    mask = np.where(np.isnan(x_array) | np.isnan(y_array), np.nan, 0)
    x_array += mask
    y_array += mask

    n = len(y_array)

    # We compute the rolling means
    x_mean_calc = np.array([np.nanmean(x_array[i:i + window]) for i in range(len(x_array) - window + 1)])
    x_mean = np.append(np.zeros(window - 1) + np.nan, x_mean_calc)
    y_mean_calc = np.array([np.nanmean(y_array[i:i + window]) for i in range(len(y_array) - window + 1)])
    y_mean = np.append(np.zeros(window - 1) + np.nan, y_mean_calc)

    # We get the mean deviations for each variable
    x = x_array - x_mean
    y = y_array - y_mean

    # We compute the square of the deviations of X
    sqr_x = x ** 2

    # The numerator of the slope
    num = x * y

    # We compute the rolling sum for the numerator
    roll_num_calc = np.array([np.nansum(num[i:i + window]) for i in range(len(x_array) - window + 1)])
    roll_num = np.append(np.zeros(window - 1) + np.nan, roll_num_calc)

    # The rolling sum for the denominator
    roll_den_calc = np.array([np.nansum(sqr_x[i:i + window]) for i in range(len(x_array) - window + 1)])
    roll_den = np.append(np.zeros(window - 1) + np.nan, roll_den_calc)

    # We ensure the denominator values differ from zero
    roll_den = np.where(roll_den == 0, 1, roll_den)

    slope = np.zeros(n)
    intercept = np.zeros(n)
    y_predict = np.zeros(n)

    for i in range(window, n):
        slope[i] = (roll_num[i] / roll_den[i]) * x[i]
        # slope[i] = np.nansum(x[i:i + window] * y[i:i + window]) / np.nansum(sqr_x[i:i + window])
        intercept[i] = y_mean[i] - slope[i] * x_mean[i]
        y_predict[i] = intercept[i] + slope[i] * x_mean[i]

    y_predict[:window] = np.nan

    return y_predict


### Rolling Max

In [12]:
@njit
def rolling_max(arr: np.array, window: int):
    """
    rolling_max stands the maximum value of a price series within a moving window
    :param arr: np.array with dimension 1, should be np.float32 numbers.
    :param window:int value, its the length of the moving window, minimum value for the window may be 2.
    :return: np.array vector with the values for the roll_max.
    """
    ma_values_close = np.array([arr[i:i + window].max() for i in range(len(arr) - window + 1)])
    roll_max = np.append(np.zeros(window - 1) + np.nan, ma_values_close).astype(np.float32)
    return roll_max

### Rolling min

In [13]:
# @njit('float32[:](float32[:], int32)', cache=True, nogil=True)
@njit
def rolling_min(arr: np.array, window: int):
    """
    rolling_min stands the minimum value of a price series within a moving window
    :param arr: np.array with dimension 1, should be np.float32 numbers.
    :param window:int value, its the length of the moving window, minimum value for the window may be 2.
    :return: np.array vector with the values for the roll_min.
    """
    ma_values_close = np.array([arr[i:i + window].min() for i in range(len(arr) - window + 1)])
    roll_min = np.append(np.zeros(window - 1) + np.nan, ma_values_close).astype(np.float32)
    return roll_min



## Squeeze Momentum Indicator Construction

### Parameters setup

In [14]:
# parameter setup
window_sma = 20
window_bb = 20
mult_bb = 2
window_kc = 20
mult_kc = 1.5
window_atr_kc = 10
window_atr = 14
window_momentum = 20

In [15]:
# Bollinger Bands
bbands = bband(Close, window_bb, mult_bb)

upper_bband = bbands[0]
lower_bband = bbands[1]
midline_bband = bbands[2]

In [16]:
# Keltner Channels
kc = keltner_channel(High, Low, Close, window_kc, mult_kc, window_atr_kc)

midline_kc = bbands[0]
upper_kc = bbands[1]
lower_kc = bbands[2]

In [17]:
# Squeeze On/Off Dots
squeeze_on = np.zeros(len(Close), dtype='bool')
squeeze_off = np.zeros(len(Close), dtype='bool')
no_squeeze = np.zeros(len(Close), dtype='bool')

for i in range(1, len(Close)):
        squeeze_on[i] = ((lower_bband[i] > lower_kc[i]) & (upper_bband[i] < upper_kc[i]))
        squeeze_off[i] = ((lower_bband[i] < lower_kc[i]) & (upper_bband[i] > upper_kc[i]))
        no_squeeze[i] = squeeze_on[i] == False & squeeze_off[i] == False

#squeeze_on
#squeeze_off
#no_squeeze

In [18]:
# Momentum Histogram

## First, calculate the Donchian midline for the specified number of momentum periods (20 is used by default):
highest_high = rolling_max(High, window_momentum)
lowest_low = rolling_min(Low, window_momentum)
donchian_midline = (highest_high + lowest_low) / 2


In [19]:
# Second, calculate the SMA of the close for the specified number of momentum periods (so by default, a 20-period SMA of price).
Close_sma = ma(Close, window_sma)

In [20]:
# Third, calculate the delta between the close and the average of the Donchian midline and SMA values using the following formula:
squeeze_momentum_values = Close - ((donchian_midline + Close_sma) / 2)
squeeze_momentum_values_lag = lag(squeeze_momentum_values, 1)

In [21]:
# Finally, use linear regression on the delta values to smooth them.
lin_squeeze_momentum_values = lin_reg(squeeze_momentum_values, squeeze_momentum_values_lag, window_kc)


#### Squeeze Momentum Function

In [22]:
@njit
def squeeze_momentum (High, Low, Close, window_bb, mult_bb, window_kc, mult_kc, 
                      window_atr_kc, window_momentum, window_sma):
    
    # Bollinger Bands
    bbands = bband(Close, window_bb, mult_bb)
    upper_bband = bbands[0]
    lower_bband = bbands[1]
    midline_bband = bbands[2]
    
    # Keltner Channels
    kc = keltner_channel(High, Low, Close, window_kc, mult_kc, window_atr_kc)
    midline_kc = bbands[0]
    upper_kc = bbands[1]
    lower_kc = bbands[2]
    
    # Momentum Histogram
    highest_high = rolling_max(High, window_momentum)
    lowest_low = rolling_min(Low, window_momentum)
    donchian_midline = (highest_high + lowest_low) / 2
    Close_sma = ma(Close, window_sma)
    squeeze_momentum_values = Close - ((donchian_midline + Close_sma) / 2)
    squeeze_momentum_values_lag = lag(squeeze_momentum_values, 1)
    lin_squeeze_momentum_values = lin_reg(squeeze_momentum_values, squeeze_momentum_values_lag, window_kc)
    
    # Squeeze On/Off Dots
    squeeze_on = np.zeros(len(Close), dtype='bool')
    squeeze_off = np.zeros(len(Close), dtype='bool')
    no_squeeze = np.zeros(len(Close), dtype='bool')

    for i in range(1, len(Close)):
        squeeze_on[i] = ((lower_bband[i] > lower_kc[i]) & (upper_bband[i] < upper_kc[i]))
        squeeze_off[i] = ((lower_bband[i] < lower_kc[i]) & (upper_bband[i] > upper_kc[i]))
        no_squeeze[i] = ((squeeze_on[i] == False) & (squeeze_off[i] == False))
        
    return lin_squeeze_momentum_values, squeeze_on, squeeze_off, no_squeeze



# Graphic Validation

In [23]:
# init bokeh
output_notebook()

# Candlestick
inc = Close > Open
dec = Close < Open

candle_size = 150

w = (17 * 30 * 30 * 25) * candle_size

is_data = pd.DataFrame(tsla_data)

is_data['date'] = is_data.index

is_data.columns= is_data.columns.str.lower()

is_data = is_data[['date', 'open', 'high', 'low', 'close', 'volume']]


lin_squeeze_momentum_values, squeeze_on, squeeze_off, no_squeeze = squeeze_momentum (High, Low, Close, 
                                                                                     window_bb, mult_bb, 
                                                                                     window_kc, mult_kc,
                                                                                     window_atr_kc, window_momentum, 
                                                                                     window_sma)



is_data['lin_squeeze_momentum_values'] = lin_squeeze_momentum_values
is_data['squeeze_on'] = squeeze_on
is_data['squeeze_off'] = squeeze_off
is_data['no_squeeze'] = no_squeeze

# print(is_data)

source_is_data = ColumnDataSource(is_data)

crosshair = CrosshairTool(dimensions='both')


p = figure(x_axis_type="datetime", plot_height=500, plot_width=1000,
           title=f'Symbol: TSLA, '
                 f'Timeframe: 1D ')


bar_high_low = p.segment("date", "high", "date", "low", color="black", source=source_is_data)
bar_high_low = p.segment("date", "high", "date", "low", color="black", source=source_is_data)
bar_open = p.vbar(is_data.date[inc], w, is_data.open[inc], is_data.close[inc], fill_color="green",
                  line_color="green")
bar_close = p.vbar(is_data.date[dec], w, is_data.open[dec], is_data.close[dec], fill_color="red",
                   line_color="red")

# Axis
p.xaxis.axis_label = 'TIME'
p.yaxis.axis_label = 'PRICE'

p_aux = figure(x_axis_type="datetime", plot_height=350, plot_width=1000,
           title= 'Squeeze momentum indicator', x_range=p.x_range)

color_green = np.zeros(len(Close), dtype='bool')
color_lime = np.zeros(len(Close), dtype='bool')
color_maroon = np.zeros(len(Close), dtype='bool')
color_red = np.zeros(len(Close), dtype='bool')

for i in range(0, len(Close)):
    green_condition = is_data['lin_squeeze_momentum_values'][i] >= 0
    lime_condition = is_data['lin_squeeze_momentum_values'][i] > is_data['lin_squeeze_momentum_values'][i-1]
    red_condition = is_data['lin_squeeze_momentum_values'][i] < is_data['lin_squeeze_momentum_values'][i-1]
#     print(green_condition, lime_condition, red_condition)
    if green_condition and lime_condition:
        color_lime[i] = True
    elif green_condition:
        color_green[i] = True
    elif not green_condition and red_condition:
        color_red[i] = True
    else:
        color_maroon[i] = True

# for i in range(0, len(Close)):
#     print(color_green[i])


bar_green = p_aux.vbar(x=is_data.date[color_green], top=is_data.lin_squeeze_momentum_values[color_green], bottom = 0, width=1, fill_color="green",
                  line_color="green")
bar_lime = p_aux.vbar(x=is_data.date[color_lime], top=is_data.lin_squeeze_momentum_values[color_lime], bottom = 0, width=1, fill_color="lime",
                  line_color="lime")
bar_maroon = p_aux.vbar(x=is_data.date[color_maroon], top=is_data.lin_squeeze_momentum_values[color_maroon], bottom = 0, width=1, fill_color="maroon",
                  line_color="maroon")
bar_red = p_aux.vbar(x=is_data.date[color_red], top=is_data.lin_squeeze_momentum_values[color_red], bottom = 0, width=1, fill_color="red",
                  line_color="red")

g = gridplot([[p], [p_aux]], sizing_mode='scale_width')
show(g)

Loading BokehJS ...

# Version 2

In [78]:
# parameter setup
length = 20
mult_bb = 2.0
lengthKC = 20
multKC = 1.5

## Squeeze Momentum Indicator by LazyBear

In [87]:
@njit
def squeeze_momentum_lazybear (High, Low, Close, length, mult_bb, lengthKC, mult_kc):
    
    # Bollinger Bands
    bbands = bband(Close, length, mult_bb)
    upper_bband = bbands[0]
    lower_bband = bbands[1]
    
    # Keltner Channels  
    kc = keltner_channel(High, Low, Close, lengthKC, mult_kc, lengthKC)
    midline_kc = bbands[0]
    upper_kc = bbands[1]
    lower_kc = bbands[2]
      
    # Momentum Histogram
    highest_high = rolling_max(High, length)
    lowest_low = rolling_min(Low, length)
    donchian_midline = (highest_high + lowest_low) / 2
    Close_sma = ma(Close, length)
    squeeze_momentum_values = Close - ((donchian_midline + Close_sma) / 2)
    
    squeeze_momentum_values_lag = lag(squeeze_momentum_values, 1)
    lin_squeeze_momentum_values = lin_reg(squeeze_momentum_values, squeeze_momentum_values_lag, length)
    
    # Squeeze On/Off Dots
    squeeze_on = np.zeros(len(Close), dtype='bool')
    squeeze_off = np.zeros(len(Close), dtype='bool')
    no_squeeze = np.zeros(len(Close), dtype='bool')

    for i in range(1, len(Close)):
        squeeze_on[i] = ((lower_bband[i] > lower_kc[i]) & (upper_bband[i] < upper_kc[i]))
        squeeze_off[i] = ((lower_bband[i] < lower_kc[i]) & (upper_bband[i] > upper_kc[i]))
        no_squeeze[i] = ((squeeze_on[i] == False) & (squeeze_off[i] == False))
        
    return lin_squeeze_momentum_values, squeeze_on, squeeze_off, no_squeeze



## Graphic Validation 2

In [88]:
# init bokeh
output_notebook()

# Candlestick
inc = Close > Open
dec = Close < Open

candle_size = 150

w = (17 * 30 * 30 * 25) * candle_size

is_data = pd.DataFrame(tsla_data)

is_data['date'] = is_data.index

is_data.columns= is_data.columns.str.lower()

is_data = is_data[['date', 'open', 'high', 'low', 'close', 'volume']]


lin_squeeze_momentum_values, squeeze_on, squeeze_off, no_squeeze = squeeze_momentum_lazybear(High, Low, Close,
                                                                                             length, mult_bb,
                                                                                             lengthKC, mult_kc)



is_data['lin_squeeze_momentum_values'] = lin_squeeze_momentum_values
is_data['squeeze_on'] = squeeze_on
is_data['squeeze_off'] = squeeze_off
is_data['no_squeeze'] = no_squeeze

# print(is_data)

source_is_data = ColumnDataSource(is_data)

crosshair = CrosshairTool(dimensions='both')


p = figure(x_axis_type="datetime", plot_height=500, plot_width=1000,
           title=f'Symbol: TSLA, '
                 f'Timeframe: 1D ')


bar_high_low = p.segment("date", "high", "date", "low", color="black", source=source_is_data)
bar_high_low = p.segment("date", "high", "date", "low", color="black", source=source_is_data)
bar_open = p.vbar(is_data.date[inc], w, is_data.open[inc], is_data.close[inc], fill_color="green",
                  line_color="green")
bar_close = p.vbar(is_data.date[dec], w, is_data.open[dec], is_data.close[dec], fill_color="red",
                   line_color="red")

# Axis
p.xaxis.axis_label = 'TIME'
p.yaxis.axis_label = 'PRICE'

p_aux = figure(x_axis_type="datetime", plot_height=350, plot_width=1000,
           title= 'Squeeze momentum indicator', x_range=p.x_range)

color_green = np.zeros(len(Close), dtype='bool')
color_lime = np.zeros(len(Close), dtype='bool')
color_maroon = np.zeros(len(Close), dtype='bool')
color_red = np.zeros(len(Close), dtype='bool')

for i in range(0, len(Close)):
    green_condition = is_data['lin_squeeze_momentum_values'][i] >= 0
    lime_condition = is_data['lin_squeeze_momentum_values'][i] > is_data['lin_squeeze_momentum_values'][i-1]
    red_condition = is_data['lin_squeeze_momentum_values'][i] < is_data['lin_squeeze_momentum_values'][i-1]
#     print(green_condition, lime_condition, red_condition)
    if green_condition and lime_condition:
        color_lime[i] = True
    elif green_condition:
        color_green[i] = True
    elif not green_condition and red_condition:
        color_red[i] = True
    else:
        color_maroon[i] = True

# for i in range(0, len(Close)):
#     print(color_green[i])


bar_green = p_aux.vbar(x=is_data.date[color_green], top=is_data.lin_squeeze_momentum_values[color_green], bottom = 0, width=1, fill_color="green",
                  line_color="green")
bar_lime = p_aux.vbar(x=is_data.date[color_lime], top=is_data.lin_squeeze_momentum_values[color_lime], bottom = 0, width=1, fill_color="lime",
                  line_color="lime")
bar_maroon = p_aux.vbar(x=is_data.date[color_maroon], top=is_data.lin_squeeze_momentum_values[color_maroon], bottom = 0, width=1, fill_color="maroon",
                  line_color="maroon")
bar_red = p_aux.vbar(x=is_data.date[color_red], top=is_data.lin_squeeze_momentum_values[color_red], bottom = 0, width=1, fill_color="red",
                  line_color="red")

g = gridplot([[p], [p_aux]], sizing_mode='scale_width')
show(g)

Loading BokehJS ...

In [89]:
is_data.tail()

,date,open,high,low,close,volume,lin_squeeze_momentum_values,squeeze_on,squeeze_off,no_squeeze
Date,,,,,,,,,,
2021-10-21,2021-10-21,856.000000,900.000000,855.500000,894.000000,31481500,38.664893,False,True,False
2021-10-22,2021-10-22,895.500000,910.000000,890.960022,909.679993,22836800,41.204501,False,True,False
2021-10-25,2021-10-25,950.530029,1045.020020,944.200012,1024.859985,62852100,47.161160,False,True,False
2021-10-26,2021-10-26,1024.689941,1094.939941,1001.440002,1018.429993,62346200,52.619547,False,True,False
2021-10-27,2021-10-27,1039.660034,1070.589966,1033.013916,1048.290039,25804890,59.106702,False,True,False
